<a href="https://colab.research.google.com/github/Ajay-2007/deep-learning-projects/blob/master/4.handwriten_digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
!pwd

/content


In [3]:
train_dataset = datasets.MNIST(root='./content', 
                               train=True,
                               transform= transforms.ToTensor(),
                               download= True)

test_dataset = datasets.MNIST(root='./content',
                               train= False,
                               transform= transforms.ToTensor(),
                               download=False)

Extracting ./content/MNIST/raw/train-images-idx3-ubyte.gz to ./content/MNIST/raw


Extracting ./content/MNIST/raw/train-labels-idx1-ubyte.gz to ./content/MNIST/raw



Extracting ./content/MNIST/raw/t10k-images-idx3-ubyte.gz to ./content/MNIST/raw


Extracting ./content/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./content/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [4]:
type(train_dataset)

torchvision.datasets.mnist.MNIST

In [5]:
train_dataset.classes

['0 - zero',
 '1 - one',
 '2 - two',
 '3 - three',
 '4 - four',
 '5 - five',
 '6 - six',
 '7 - seven',
 '8 - eight',
 '9 - nine']

In [6]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./content
    Split: Train
    StandardTransform
Transform: ToTensor()

In [0]:
input_size = 784
hidden_size = 400
out_size = 10
epochs = 10
batch_size = 100
learning_rate = 0.001 

In [0]:
# Make Data iterable by loading it to a loader.
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [9]:
 class Net(nn.Module):
   def __init__(self, input_size, hidden_size, out_size):
     super(Net, self).__init__()
     self.fc1 = nn.Linear(input_size, hidden_size)
     self.relu = nn.ReLU()
     self.fc2 = nn.Linear(hidden_size, hidden_size)
     self.fc3 = nn.Linear(hidden_size, out_size)
     
   def forward(self, x):
     out = self.fc1(x)
     out = self.relu(out)
     out = self.fc2(out)
     out = self.relu(out)
     out = self.fc3(out)
     return out

In [0]:
net = Net(input_size, hidden_size, out_size)
CUDA = torch.cuda.is_available()
if CUDA:
  net = net.cuda()

# The Loss function. The Cross Entropy Loss comes along with softmax.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [11]:
# list(net.parameters())
net.parameters

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=400, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=10, bias=True)
)>

In [12]:
# Visualize the train loader
for i, (image, labels) in enumerate(train_loader):
  print(image.size())
  image = image.view(-1, 784)
  print(image.size())
  break

torch.Size([100, 1, 28, 28])
torch.Size([100, 784])


In [13]:
correct_train = 0
total_train = 0

for epoch in range(epochs):
  for i, (image, labels) in enumerate(train_loader):
    # Flatten the image from size (batch, 1, 28, 28) --> (100, 1, 28, 28) where 1
    # represents the number of channels (grayscale-->1),
    # to size (100, 784) and wrap it in a variable
    images = Variable(image.view(-1, 28*28))
    labels = Variable(labels)
    if CUDA:
      images = images.cuda()
      labels = labels.cuda()
    # clear the param_grad in param = param - lr*param_grad, so it won't be accumulated
    optimizer.zero_grad()
    outputs = net.forward(images) # Forward Propogation
    _, predicted = torch.max(outputs.data, 1) # Return the second argument of torch.max
                                              # Which represents the index location of each
                                              # maximum value found
    total_train += labels.size(0)
    if CUDA:
      correct_train += (predicted.cpu() == labels.cpu()).sum()
    else:
      correct_train += (predicted == labels)
    
    loss = criterion(outputs, labels) # Difference between the acutal and predicted (loss function)
    loss.backward() # Backpropagation
    optimizer.step()  # Update the weights
    # print(loss.item())
    # print(loss)
    if (i+1) % 100 == 0:
      print('Epoch  [{}/{}], Iteration [{}/{}], Training Loss: {}, Training Accuracy: {}%'\
            .format(epoch+1, epochs, i+1, len(train_dataset)//batch_size, loss.item(),
            (100*correct_train/total_train))) 
    # break
print('DONE TRAINING')

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Epoch  [1/10], Iteration [100/600], Training Loss: 0.3017217218875885, Training Accuracy: 82%
Epoch  [1/10], Iteration [200/600], Training Loss: 0.2350020855665207, Training Accuracy: 87%
Epoch  [1/10], Iteration [300/600], Training Loss: 0.1342708021402359, Training Accuracy: 89%
Epoch  [1/10], Iteration [400/600], Training Loss: 0.11525962501764297, Training Accuracy: 90%
Epoch  [1/10], Iteration [500/600], Training Loss: 0.10644671320915222, Training Accuracy: 91%
Epoch  [1/10], Iteration [600/600], Training Loss: 0.1636093258857727, Training Accuracy: 92%
Epoch  [2/10], Iteration [100/600], Training Loss: 0.05840526521205902, Training Accuracy: 92%
Epoch  [2/10], Iteration [200/600], Training Loss: 0.13707520067691803, Training Accuracy: 93%
Epoch  [2/10], Iteration [300/600], Training Loss: 0.06679558008909225, Training Accuracy: 93%
Epoch  [2/10], Iteration [400/600], Training Loss: 0.06693119555711746, Training Accuracy: 94%
Epoch  [2/10], Iteration [500/600], Training Loss: 0.2

In [15]:
correct = 0
total = 0
for i, (images, labels) in enumerate(test_loader):
  images = Variable(images.view(-1, 28*28))
  if CUDA:
    images = images.cuda()
  # For each input (sample/row) in the batch, the output will contain 10 elements
  outputs = net.forward(images)
  # We could also write: predicted = outputs.data.max(1)[1]
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0) # Increment the total count (100)
  # We can also use: correct += predicted.eq(labels).sum()
  if CUDA:
    correct += (predicted.cpu() == labels.cpu()).sum()
  else:
    correct += (predicted == labels).sum()

print('Final Test Accuracy: {:.2f}'.format(100 * correct/total))

Final Test Accuracy: 97.00
